In [1]:
#include <iostream>
#include <iomanip>   // for setw and setfill stream manipulators
#include <stdexcept> // for invalid_argument exception class
#include <sstream>   // for ostringstream class
#include <string>
using namespace std;

# Classes: Advanced

- Friend function and classes
- **this** pointer
- Pointer to a private data member
- Default memberwise assignment
- Const objects
- Const member functions
- Composition
- Default copy constructor
- Static class members
- Dynamic Memory Management

## Friend Functions and Classes

A **friend function** of a class is a non-member function that has the right to access the public and non-public class members.
- Standalone functions, entire classes or member functions of other classes may be declared to be friends of another class, see [C++ Reference Manual](https://en.cppreference.com/w/cpp/language/friend).

Declaring a friend
- To declare a non-member function as a friend of a class, place the function prototype in the class definition and precede it with keyword **friend**.
- To declare all member functions of a class as friends of other class, place a **friend** declaration of the form
```cpp
friend class <frend-class-name>;
// or
friend <frend-function-name>;
```

In [2]:
class Point3D {
    friend void setX(Point3D&, int); // friend function declaration
    friend class Point3DUtils;       // friend class declaration
    friend ostream& operator<<(ostream&, const Point3D&); // friend streaming operator
private:    
    int x; int y; int z;
public:
    Point3D(int r = 0, int s = 0, int t = 0): x(r), y(s), z(t) {}    
    string toString() const {
        ostringstream out;
        out << "(x:" << x << ", y:" << y  << ", z:" << z << ")";
        return out.str();
    }    
}

*Note:* **Member access** notions of private, protected and public are **not relevant to friend declarations**, so friend declarations can be placed **anywhere** in a class definition.

In [11]:
// friend function that has access to
// Point3D private data members
void setX(Point3D& p, int x){
    p.x = x;
}

In [12]:
// friend function that has access to
// Point3D private data members
void setY(Point3D& p, int x){
    p.x = x;
}

input_line_19:4:7: error: 'x' is a private member of '__cling_N510::Point3D'
    p.x = x;
      ^
input_line_17:6:9: note: declared private here
    int x; int y; int z;
        ^


Interpreter Error: 

In [13]:
{
    Point3D p;
    cout << p.toString() << endl;
    setX(p, 10);
    cout << p.toString() << endl;
}

(x:0, y:0, z:0)
(x:10, y:0, z:0)


In [14]:
// friend class that has access to
// Point3D private data members
class Point3DUtils {
    int defaultx;
public:
    Point3DUtils(): defaultx(1000) {}
    void setDefaultX(Point3D& p){ p.x = defaultx;}
    void setDefaultY(Point3D& p){ p.y = defaultx+1;}
}

In [15]:
{
    Point3DUtils pu;
    Point3D p;
    cout << p.toString() << endl;
    pu.setDefaultX(p);
    cout << p.toString() << endl;
    pu.setDefaultY(p);
    cout << p.toString() << endl;
}

(x:0, y:0, z:0)
(x:1000, y:0, z:0)
(x:1000, y:1001, z:0)


In [ ]:
ostream& operator<<(ostream& out, const Point3D& p){
    out << "(" << p.x << p.y << p.z << ")" << endl; // p.x, p.y, p.z are private
    return out;
}

In [ ]:
{
    Point3D p;
    cout << p << endl;
}

## Friendship

Friendship is granted, not taken
- for class B to be a friend of class A, class A must explicitly declare that class B is its friend.

Friendship is not symmetric
- if class A is a friend of class B, you cannot infer that class B is a friend of class A.

Friendship is not transitive
- if class A is a friend of class B and class B is a friend of class C, you cannot infer that class A is a friend of class C.

## this Pointer

Every object has access to its own address through a pointer called **this** (a C++ keyword).
- The **this** pointer is not part of the object itself
    - the memory occupied by the this pointer is not reflected in the result of a **sizeof** operation on the object.
- Rather, the this pointer is passed (by the compiler) as an **implicit** argument to each of the object's non-static member functions.

Member functions use the this pointer **implicitly** or **explicitly** to reference an object's data members and other member functions.
- A common explicit use of the this pointer is to avoid naming conflicts between a class's data members and member-function parameters (or other local variables).

In [13]:
class Point {
    int x;
public:
    Point(int val) : x{val} {}
    int getX() { return x; }
    void setX(int x){
        // implicitly use the this pointer to access the member x
        cout << "[setX] x = " << x << endl;
        // explicitly use the this pointer and the arrow operator
        // to access the member x
        this->x = x;
        // explicitly use the dereferenced this pointer and
        // the dot operator to access the member x
        cout << "(*this).x = " << (*this).x << endl;
    }
}

In [14]:
{
    Point p{0};
    cout << "After creation: p.x = " << p.getX() << endl;
    p.setX(2);
    cout << "p.x = " << p.getX() << endl;
}

After creation: p.x = 0
[setX] x = 2
(*this).x = 2
p.x = 2


## Pointer to a Private Data Member

A reference to an object is an alias for the name of the object and, hence, may be used on the left side of an assignment statement.
- The reference makes a perfectly acceptable **lvalue** that can receive a value.

A member function can return a reference to a private data member of that class.
- Returning a reference or a pointer to a private data member breaks the encapsulation of the class.
- Makes the client code dependent on the representation of the class’s data.
- <span style="color:red">NEVER DO THAT!!!</span>


In [15]:
class PrivateRef {
    int x;
public:
    int getValue() const { return x; };
    void setValue(int v){ x=v; };
    int& setValueBad(int v){ x=v; return x; };
}

In [19]:
{
    PrivateRef X;
    cout << "X: " << X.getValue() << endl;
    X.setValue(10);
    cout << "X: " << X.getValue() << endl;
    X.setValueBad(20) = 30; // BIG PROBLEM
    cout << "X: " << X.getValue() << endl;
}

X: -1147710480
X: 10
X: 30


## Default Memberwise Assignment

- The **assignment operator (=)** can be used to **assign an object to another object** of the same type.
    - By default, such assignment is performed by **memberwise assignment** (also called *copy assignment*).    
- Each data member of the object on the right of the assignment operator is assigned individually to the same data member in the object on the left of the assignment operator.

<span style="color:red">Caution: Memberwise assignment can cause serious problems when used with a class whose data members contain pointers to dynamically allocated memory.</span>

In [14]:
class Point2D {
    int x;
    int y;
public:
    Point2D(int v = 0, int w = 0): x(v), y(w){}
    ~Point2D(){ cout << "Point2D object [" << toString() <<"] is destroyed" << endl; }
    
    void setPoint(int v, int w) {x= v; y = w; }; 
    string toString() const {
        ostringstream out;
        out << "(x:" << x << ", y:" << y << ")";
        return out.str();
    }
}

In [15]:
{
    Point2D p1{3,2};
    Point2D p2;
    cout << "Before: p1: " << p1.toString() << ", p2: " << p2.toString() << endl;
    p2 = p1; //  memberwise assignment
    cout << " After: p1: " << p1.toString() << ", p2: " << p2.toString() << endl;
}

Before: p1: (x:3, y:2), p2: (x:0, y:0)
 After: p1: (x:3, y:2), p2: (x:3, y:2)
Point2D object [(x:3, y:2)] is destroyed
Point2D object [(x:3, y:2)] is destroyed


## Const Objects

- Attempts to modify a const object are **caught at compile time** rather than causing execution-time errors.
- Declaring variables and objects **const** when appropriate can improve performance.

In [16]:
{
    Point2D p1{1,2};
    const Point2D p2;
    cout << "p1: " << p1.toString() << ", p2: " << p2.toString() << endl;
    p2 = p1; //  memberwise assignment
}

input_line_23:6:8: error: no viable overloaded '='
    p2 = p1; //  memberwise assignment
    ~~ ^ ~~
input_line_21:1:7: note: candidate function (the implicit copy assignment operator) not viable: 'this'
      argument has type 'const __cling_N514::Point2D', but method is not marked
      const
class Point2D {
      ^


Interpreter Error: 

## Const Member Functions

- C++ disallows member function calls for **const** objects unless the member functions themselves are also declared **const**.
    - This is true even for get member functions that do not modify the object.
    - This is also a key reason that we've declared as **const** all member-functions that do not modify the objects on which they're called.
- Constructor **must be a non-const** member function, but it can still be used to initialize a **const** object
    - Attempting to declare a constructor or destructor const is a compilation error.

In [17]:
{
    Point2D p1{1,2};
    const Point2D p2; 
                      // OBJECT    MEMBER
    p1.setPoint(1,1); // non-const non-const
    p1.toString();    // non-const const
    p2.setPoint(1,1); // const     non-const  !!! PROBLEM
    p2.toString();    // const     const
}

input_line_24:8:5: error: member function 'setPoint' not viable: 'this' argument has type 'const
      __cling_N514::Point2D', but function is not marked const
    p2.setPoint(1,1); // const     non-const  !!! PROBLEM
    ^~
input_line_21:8:10: note: 'setPoint' declared here
    void setPoint(int v, int w) {x= v; y = w; }; 
         ^


Interpreter Error: 

## Composition: Objects as Members of Classes

It is natural to think of objects as containing other objects.
- an aircraft may be studied by decomposing it into its propulsion system, etc.
- object structure
- "part-of" hierarchy / "has-a" relation
    - has-a relationship - a class can have objects of other classes as members.

Data members are constructed in the order in which they're declared in the class definition
- before their enclosing class objects are constructed.

In [18]:
class Location {
    string name;   // composition: member object
    Point2D coord; // composition: member object
public:
    Location(const string& n, int x, int y) : name(n), coord{x,y} {}   // using Point2D constructor
    Location(const string& n, const Point2D &p) : name(n), coord{p} {} // using Point2D copy constructor
    ~Location(){ cout << "Location object [" << toString() <<"] is destroyed" << endl; }
    string toString() const {
        ostringstream out;
        out << name << " @ " << coord.toString();
        return out.str();
    }    
    void setLocation(int x, int y) { coord.setPoint(x,y); }
}

In [19]:
{
    Point2D p1;
}

Point2D object [(x:0, y:0)] is destroyed


In [9]:
{
    Location home{"home", 1, 2};
    cout << home.toString() << endl;    
}

home @ (x:1, y:2)
Location object [home @ (x:1, y:2)] is destroyed
Point2D object [(x:1, y:2)] is destroyed


## Default Copy Constructor

Objects may be passed as function arguments and may be returned from functions.
- such passing and returning is performed using pass-by-value by default
    - a copy of the object is passed or returned. 
- C++ creates a **new object** and uses a **copy constructor** to copy the original object's values into the new object.
    - the compiler provides a default copy constructor that copies each member of the original object into the corresponding member of the new object.

The member initializers specify **Location** constructor parameters being passed to the constructors of the **Point2D** data members.
- If a member object is not initialized through a member initializer, the member object's default constructor will be called implicitly.

In [10]:
{
    cout << "Begining of local scope" << endl;
    Point2D p{2,3};
    Location home{"home", p};
    cout << home.toString() << endl;    
    home.setLocation(3,4);
    cout << home.toString() << endl;    
    cout << "End of local scope" << endl;
}

Begining of local scope
home @ (x:2, y:3)
home @ (x:3, y:4)
End of local scope
Location object [home @ (x:3, y:4)] is destroyed
Point2D object [(x:3, y:4)] is destroyed
Point2D object [(x:2, y:3)] is destroyed


## Cascaded Function Calls

We can use of the **this** pointer is to enable **cascaded member-function calls** by returning **\*this**.
- invoking multiple functions sequentially in the same statement

In [5]:
class Coord {
    int x; int y;
public:
    Coord& setX(int x) { this->x = x; return *this ;}
    Coord& setY(int y) { this->y = y; return *this ;}
    void print() const { cout << "(x:" << x << ", y:" << y  << ")" << endl; }
}

In [6]:
{
    Coord c;
    c.print();
    c.setX(1).setY(2);
    c.print();
}

(x:1799847936, y:32557)
(x:1, y:2)


## Static Class Members

- In certain cases, only one copy of a variable should be **shared** by **all** objects of a class.
- A ***static* data member** is used for these and other reasons.
- Such a variable represents "classwide" information,
    - data that is shared by all instances and is not specific to any one object of the class.    
- Static data members have **class scope**.
- A static data member must be initialized **exactly once**.
- Fundamental-type static data members are initialized by default to 0.
- In C++11's all static const data members can have in-class initializers.

In [ ]:
// Counter.h
#include <iostream>
using namespace std;
class Counter {
    static unsigned int count; // number of objects instantiated
public:
    Counter()  { cout << "new object created" << endl; ++count; };
    ~Counter() { cout << "object destroyed" << endl; --count; };
    static unsigned int getCount();
}

In [ ]:
// main.cpp
#include <iostream>
#include "Counter.h"
using namespace std;
// define and initialize static data member at global namespace scope
unsigned int Counter::count{100}; // cannot include keyword static
// define static member function that returns number of initialized objects
unsigned int Counter::getCount(){ return count; }
int main() {
    cout << "Counter starts with " << Counter::getCount() << endl;
    {
        Counter c1;
        cout << "Counter after 'c1' created: " << Counter::getCount() << endl;
        {
            Counter c2;
            cout << "Counter after 'c2' created: " << Counter::getCount() << endl;
        }
        cout << "Counter after 'c2' destroyed: " << Counter::getCount() << endl;
    }
    cout << "Counter after 'c1' destroyed:  " << Counter::getCount() << endl;
}

In [16]:
class C {
public:
    int x{-1};
    void setX(int x){
        x = x;
    }
}

In [18]:
{
    C obj;
    cout << obj.x << endl;
    obj.setX(10);
    cout << obj.x << endl;
}

-1
-1


## Dynamic Memory Management

- You can control memory **allocation** and **deallocation** in a program for objects and for arrays of any built-in or user-defined type.
    - Known as **dynamic memory management**; performed with **new** and **delete**.
- Use the **new** operator to dynamically allocate the exact amount of memory required to hold an object at execution time.
- The object is created in the free store - **heap** - a region of memory assigned to each program for storing dynamically allocated objects.
- Once memory is allocated, you can access it via the pointer that operator **new** returns.
- Return memory by using the **delete** operator to **deallocate** it.

## Obtaining Dynamic Memory

- The **new** operator allocates storage of the proper size for an object
    - calls the default constructor to initialize the object, and
    - returns a pointer to the type specified to the right of the new operator.
- If **new** is unable to find sufficient space in memory for the object, it indicates that an error occurred
    - by "throwing an exception".

In [ ]:
{
    ClassWithIncOp2* ptr{new ClassWithIncOp2(7)};
    cout << ptr->getX();
}

## Releasing Dynamic Memory

- To destroy a dynamically allocated object, use the **delete** operator as follows:
        delete ptr;
- This statement first calls the **destructor** for the object to which `ptr` points,
    - then deallocates the memory associated with the object, returning the memory to the free store.

In [ ]:
{
    ClassWithIncOp2* ptr{new ClassWithIncOp2(7)};
    cout << ptr->getX() << endl;
    delete ptr;
    // delete ptr; //DO NOT DO THAT TWO TIMES
    ptr = nullptr;
    cout << ptr->getX();
}

## Tips

- **Memory leak:** Not releasing dynamically allocated memory when it's no longer needed can cause the system to run out of memory prematurely.
- Do not delete memory that was not allocated by **new**. Doing so results in undefined behavior.

- After you delete a block of dynamically allocated memory, be sure not to delete the same block again.
    - One way to guard against this is to immediately set the pointer to `nullptr`.
    - Deleting a `nullptr` has no effect.

## Initializing Dynamic Memory

- You can provide an **initializer** for a newly created fundamental type variable, as in
        double *ptr{new double{3.14159}};
- The same syntax can be used to specify a comma-separated list of arguments to the constructor of an object.
- You can also use the **new** operator to allocate built-in arrays dynamically.
        int *gradesArray{new int[10]{}};        
- A dynamically allocated array's **size** can be specified using any non-negative integral expression that can be evaluated at execution time.

## Releasing Dynamically Allocated Built-In Arrays

- To deallocate a dynamically allocated array, use the statement
        delete[] ptr;
- If the pointer points to a built-in array of objects, the statement first calls the destructor for every object in the array, then deallocates the memory.
- Using **delete** or **[]** on a **nullptr** has no effect.
- Using **delete** instead of **delete[]** for built-in arrays of objects can lead to runtime logic errors.